In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import numpy as np
import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn import model_selection, svm
import pandas as pd
from keras.utils import np_utils
from sklearn.metrics import accuracy_score, classification_report

data = open('data.txt',encoding="utf-8").read()
labels, texts = [], []

for i, line in enumerate(data.split("\n")):
    content = line.split(":")
    labels.append(content[0])
    texts.append(" ".join(content[1:]))
    


trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'],random_state=42)



encoder = preprocessing.LabelEncoder()
trainLabels = encoder.fit_transform(train_y)
trainLabels = [np_utils.to_categorical(i, num_classes=8) for i in trainLabels]
trainLabels = np.asarray(trainLabels)


validLabels = encoder.fit_transform(valid_y)
validLabels = [np_utils.to_categorical(i, num_classes=8) for i in validLabels]
validLabels = np.asarray(validLabels)

# Using FastText pre trained telugu embeddings
embeddings_index = {}

import numpy as np

from keras.preprocessing import text, sequence

for i, line in enumerate(open('/content/drive/MyDrive/te.wiki.bpe.vs200000.d300.w2v.txt',encoding="utf-8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x),maxlen=32)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x),maxlen=32)

embedding_matrix = np.zeros((len(word_index)+1, 300))

for word,i in word_index.items():
        embedding_vector = embeddings_index.get(word)    # checking that particular indexed word in telugu embedding .vec file
        if embedding_vector is not None:                 # if it is found in that .vec file  
            embedding_matrix[i] = embedding_vector 




In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((32, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    

    #Add the convolutional layer
    conv_layer = layers.Convolution1D(256, 3, activation="relu")(embedding_layer)
    
    #Add the pooling layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(128, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(8, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])

    return model


cnn = create_cnn()

cnn.fit(train_seq_x, trainLabels, epochs=10)
    
    # predict the labels on validation dataset
predictions = cnn.predict(valid_seq_x)
predictions1 = predictions.argmax(axis=-1)
validLabels1 = validLabels.argmax(axis=-1)


print(predictions1)
acc = metrics.accuracy_score(predictions1, validLabels1)

print(acc)

Epoch 1/10
32/32 [==============================] - 2s 26ms/step - loss: 1.6404 - accuracy: 0.4251
Epoch 2/10
32/32 [==============================] - 1s 29ms/step - loss: 0.6854 - accuracy: 0.8401
Epoch 3/10
32/32 [==============================] - 1s 28ms/step - loss: 0.3930 - accuracy: 0.9113
Epoch 4/10
32/32 [==============================] - 1s 25ms/step - loss: 0.2134 - accuracy: 0.9533
Epoch 5/10
32/32 [==============================] - 1s 24ms/step - loss: 0.1201 - accuracy: 0.9806
Epoch 6/10
32/32 [==============================] - 1s 28ms/step - loss: 0.0758 - accuracy: 0.9935
Epoch 7/10
32/32 [==============================] - 1s 26ms/step - loss: 0.0669 - accuracy: 0.9866
Epoch 8/10
32/32 [==============================] - 1s 26ms/step - loss: 0.0361 - accuracy: 0.9966
Epoch 9/10
32/32 [==============================] - 1s 27ms/step - loss: 0.0384 - accuracy: 0.9928
Epoch 10/10
32/32 [==============================] - 1s 27ms/step - loss: 0.0318 - accuracy: 0.9960
[3 3 3 3 

In [ ]:
#RNN Lstm glove 

def create_rnn():
    # Add an Input Layer
    input_layer = layers.Input((32, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    

    
        # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(128, activation="tanh")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(8, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])

    return model


lstm = create_rnn()

lstm.fit(train_seq_x, trainLabels, epochs=10)
    
    # predict the labels on validation dataset
predictions = lstm.predict(valid_seq_x)
predictions1 = predictions.argmax(axis=-1)
validLabels1 = validLabels.argmax(axis=-1)


print(predictions1)
acc = metrics.accuracy_score(predictions1, validLabels1)

print(acc)


Epoch 1/10
32/32 [==============================] - 5s 47ms/step - loss: 1.6958 - accuracy: 0.4896
Epoch 2/10
32/32 [==============================] - 1s 45ms/step - loss: 0.8725 - accuracy: 0.7590
Epoch 3/10
32/32 [==============================] - 2s 47ms/step - loss: 0.5481 - accuracy: 0.8523
Epoch 4/10
32/32 [==============================] - 1s 47ms/step - loss: 0.3338 - accuracy: 0.9028
Epoch 5/10
32/32 [==============================] - 1s 46ms/step - loss: 0.2104 - accuracy: 0.9446
Epoch 6/10
32/32 [==============================] - 2s 48ms/step - loss: 0.1833 - accuracy: 0.9497
Epoch 7/10
32/32 [==============================] - 2s 48ms/step - loss: 0.1120 - accuracy: 0.9695
Epoch 8/10
32/32 [==============================] - 2s 50ms/step - loss: 0.1022 - accuracy: 0.9771
Epoch 9/10
32/32 [==============================] - 2s 51ms/step - loss: 0.1004 - accuracy: 0.9725
Epoch 10/10
32/32 [==============================] - 1s 45ms/step - loss: 0.0547 - accuracy: 0.9914
[3 3 3 3 

In [ ]:
#RNN GRU glove

def create_rnngru():
    # Add an Input Layer
    input_layer = layers.Input((32, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
 

    
 # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(128, activation="tanh")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(8, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])

    return model


gru = create_rnngru()

gru.fit(train_seq_x, trainLabels, epochs=10)
    
    # predict the labels on validation dataset
predictions = gru.predict(valid_seq_x)
predictions1 = predictions.argmax(axis=-1)
validLabels1 = validLabels.argmax(axis=-1)


print(predictions1)
acc = metrics.accuracy_score(predictions1, validLabels1)

print(acc)


Epoch 1/10
32/32 [==============================] - 4s 39ms/step - loss: 1.6497 - accuracy: 0.4941
Epoch 2/10
32/32 [==============================] - 1s 42ms/step - loss: 0.7520 - accuracy: 0.7856
Epoch 3/10
32/32 [==============================] - 1s 41ms/step - loss: 0.5117 - accuracy: 0.8573
Epoch 4/10
32/32 [==============================] - 1s 42ms/step - loss: 0.3608 - accuracy: 0.9005
Epoch 5/10
32/32 [==============================] - 1s 40ms/step - loss: 0.2559 - accuracy: 0.9333
Epoch 6/10
32/32 [==============================] - 1s 41ms/step - loss: 0.1897 - accuracy: 0.9468
Epoch 7/10
32/32 [==============================] - 1s 38ms/step - loss: 0.1230 - accuracy: 0.9622
Epoch 8/10
32/32 [==============================] - 1s 43ms/step - loss: 0.1202 - accuracy: 0.9676
Epoch 9/10
32/32 [==============================] - 1s 41ms/step - loss: 0.0653 - accuracy: 0.9868
Epoch 10/10
32/32 [==============================] - 2s 78ms/step - loss: 0.0483 - accuracy: 0.9945
[3 3 6 3 

In [ ]:
#Bidirectional RNN glove

def create_brnn():
    # Add an Input Layer
    input_layer = layers.Input((32, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    

    
 # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(128, activation="tanh")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(8, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])

    return model


biRNN = create_brnn()
 
biRNN.fit(train_seq_x, trainLabels, epochs=10)
    
    # predict the labels on validation dataset
predictions = biRNN.predict(valid_seq_x)
predictions1 = predictions.argmax(axis=-1)
validLabels1 = validLabels.argmax(axis=-1)


print(predictions1)
acc = metrics.accuracy_score(predictions1, validLabels1)

print(acc)


Epoch 1/10
32/32 [==============================] - 7s 72ms/step - loss: 1.6695 - accuracy: 0.4326
Epoch 2/10
32/32 [==============================] - 2s 74ms/step - loss: 0.8329 - accuracy: 0.7772
Epoch 3/10
32/32 [==============================] - 2s 69ms/step - loss: 0.4988 - accuracy: 0.8607
Epoch 4/10
32/32 [==============================] - 2s 67ms/step - loss: 0.3611 - accuracy: 0.8922
Epoch 5/10
32/32 [==============================] - 2s 70ms/step - loss: 0.2716 - accuracy: 0.9203
Epoch 6/10
32/32 [==============================] - 2s 69ms/step - loss: 0.1873 - accuracy: 0.9520
Epoch 7/10
32/32 [==============================] - 2s 69ms/step - loss: 0.1269 - accuracy: 0.9671
Epoch 8/10
32/32 [==============================] - 2s 73ms/step - loss: 0.1013 - accuracy: 0.9787
Epoch 9/10
32/32 [==============================] - 2s 72ms/step - loss: 0.0714 - accuracy: 0.9892
Epoch 10/10
32/32 [==============================] - 2s 65ms/step - loss: 0.0579 - accuracy: 0.9908
[3 3 6 3 

In [ ]:
#RCNN


def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((32, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)


    
# Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(128, activation="tanh")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(8, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])

    return model


rcnn = create_rcnn()
 
rcnn.fit(train_seq_x, trainLabels, epochs=10)
    
    # predict the labels on validation dataset
predictions = rcnn.predict(valid_seq_x)
predictions1 = predictions.argmax(axis=-1)
validLabels1 = validLabels.argmax(axis=-1)


print(predictions1)
acc = metrics.accuracy_score(predictions1, validLabels1)

print(acc)


Epoch 1/10
32/32 [==============================] - 2s 14ms/step - loss: 1.7745 - accuracy: 0.3273
Epoch 2/10
32/32 [==============================] - 0s 13ms/step - loss: 0.9958 - accuracy: 0.7187
Epoch 3/10
32/32 [==============================] - 0s 14ms/step - loss: 0.6400 - accuracy: 0.8107
Epoch 4/10
32/32 [==============================] - 0s 13ms/step - loss: 0.4276 - accuracy: 0.8840
Epoch 5/10
32/32 [==============================] - 0s 14ms/step - loss: 0.2860 - accuracy: 0.9398
Epoch 6/10
32/32 [==============================] - 0s 13ms/step - loss: 0.2130 - accuracy: 0.9562
Epoch 7/10
32/32 [==============================] - 0s 14ms/step - loss: 0.1605 - accuracy: 0.9603
Epoch 8/10
32/32 [==============================] - 0s 14ms/step - loss: 0.1553 - accuracy: 0.9483
Epoch 9/10
32/32 [==============================] - 0s 13ms/step - loss: 0.1203 - accuracy: 0.9518
Epoch 10/10
32/32 [==============================] - 0s 13ms/step - loss: 0.1071 - accuracy: 0.9675
[0 1 3 6 

In [ ]:
input1=['భారతదేశ జనాభా ఎంత? ']
valid_seq1 = sequence.pad_sequences(token.texts_to_sequences(input1),maxlen=32)

predict=cnn.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('CNN ',encoder.inverse_transform(predict))

predict=lstm.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('LSTM ',encoder.inverse_transform(predict))

predict=gru.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('GRU ',encoder.inverse_transform(predict))

predict=biRNN.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('BI RNN ',encoder.inverse_transform(predict))

predict=rcnn.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('RCNN ',encoder.inverse_transform(predict))


CNN  ['NUMB']
LSTM  ['NUMB']
GRU  ['NUMB']
BI RNN  ['NUMB']
RCNN  ['NUMB']


In [ ]:
input1=['మధ్యప్రదేశ్ రాజధాని ఏమిటి?']
valid_seq1 = sequence.pad_sequences(token.texts_to_sequences(input1),maxlen=32)

predict=cnn.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('CNN ',encoder.inverse_transform(predict))

predict=lstm.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('LSTM ',encoder.inverse_transform(predict))

predict=gru.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('GRU ',encoder.inverse_transform(predict))

predict=biRNN.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('BI RNN ',encoder.inverse_transform(predict))

predict=rcnn.predict(valid_seq1)
predict = predict.argmax(axis=-1)
print('RCNN ',encoder.inverse_transform(predict))



CNN  ['LOCA']
LSTM  ['LOCA']
GRU  ['LOCA']
BI RNN  ['LOCA']
RCNN  ['LOCA']
